In [32]:
ID="20220226_robustness"
ID

'20220226_robustness'

In [3]:
rm -rf /data/output/$ID

In [4]:
mkdir /data/output/$ID

In [5]:
cp meta.csv /data/output/$ID

In [6]:
!cd onnxzoo2filterdb && python3 create_filter_storage.py /ssd1/robustness_onnx_zoo/ /data/output/$ID/meta.csv /data/output/$ID/ --split_after 1000 --force

100%|█████████████████████████████████████████| 119/119 [01:52<00:00,  1.06it/s]


In [7]:
!cd onnxzoo2filterdb && python3 extract_filters.py /data/output/$ID/ /data/output/$ID/datastorage.meta.csv /data/output/$ID/filterinfo.csv /data/output/$ID/

2022-03-02 11:40:55,617 [INFO]  starting extraction Namespace(dir='/data/output/20220226_robustness/', filter_info_output_file='/data/output/20220226_robustness/filterinfo.csv', filter_output_file='/data/output/20220226_robustness/', filter_shape=(3, 3), first_convs_only=False, meta_file=<_io.TextIOWrapper name='/data/output/20220226_robustness/datastorage.meta.csv' mode='r' encoding='UTF-8'>)
2022-03-02 11:40:55,621 [INFO]  loading chunk /data/output/20220226_robustness/datastorage.0000.pkl
2022-03-02 11:41:12,852 [INFO]  loaded ... processing
2022-03-02 11:41:39,980 [INFO]  finished chunk /data/output/20220226_robustness/datastorage.0000.pkl with 119 models
2022-03-02 11:41:42,221 [INFO]  writing filter info dataframe: /data/output/20220226_robustness/filterinfo.csv
2022-03-02 11:41:42,242 [INFO]  writing filters /data/output/20220226_robustness/
2022-03-02 11:42:39,845 [INFO]  finished


In [8]:
!echo "Created by processing_pipeline.ipynb on" $timestamp >> /data/output/$ID/logs/pipeline_log.txt

/bin/bash: /data/output//logs/pipeline_log.txt: No such file or directory


In [34]:
from onnxzoo2filterdb.filter_dataset import FilterDataset
import pandas as pd
import numpy as np
import os

root_path = "/data/output"
dataset_id = ID
filter_dataset = FilterDataset(dataset_id, root=root_path, filters=False)
fid_by_mid_with_meta = filter_dataset.df_filter_info.set_index([filter_dataset.df_filter_info.index, "conv_depth", "conv_depth_norm", "layer_id"]).rename(columns={"depth": "layer_depth"})
fid_by_mid_with_meta["filter_ids"] = fid_by_mid_with_meta["filter_id_start"].apply(str) + ":" + (fid_by_mid_with_meta["filter_id_end"]-1).apply(str) 
del fid_by_mid_with_meta["filter_id_start"] 
del fid_by_mid_with_meta["filter_id_end"]
fid_by_mid_with_meta = fid_by_mid_with_meta.join(filter_dataset.df_fused_meta, on=["model_id"])
fid_by_mid_with_meta.to_hdf(os.path.join(root_path, dataset_id, "dataset.h5"), "meta")

Loading tables ...
 df_filter_info
 df_meta
 df_meta_spreadsheets
 merging meta
Done


/opt/conda/lib/python3.8/site-packages/pandas/core/generic.py:2606: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->Index(['filter_ids', 'model', 'path', 'producer', 'Name', 'Paper',
       'Tracer Warning', 'Selection', 'Robust', 'Network', 'Backbone',
       'Framework', 'Pretraining-Dataset', 'Training-Dataset',
       'Visual Category', 'Visual Category_micro', 'Precision', 'Task'],
      dtype='object')]

  pytables.to_hdf(
